In [1]:
import imaplib
import subprocess
import pandas as pd
from utils import *
from os import environ

#environ['OP_SESSION_1234']='****'

imap_host = subprocess.check_output('op read op://Private/qggbe7eu3xeapptffcpq6aogku/server', shell=True).decode("utf-8").rstrip()
imap_user = subprocess.check_output('op read op://Private/qggbe7eu3xeapptffcpq6aogku/benutzername', shell=True).decode("utf-8").rstrip()
imap_pass = subprocess.check_output('op read op://Private/qggbe7eu3xeapptffcpq6aogku/passwort', shell=True).decode("utf-8").rstrip()


In [18]:
import pickle

# load pickle
cv = pickle.load(open("data/cv.pkl", "rb"))
model = pickle.load(open("data/model.pkl", "rb"))


In [19]:
# connect to host using SSL
imap = imaplib.IMAP4_SSL(imap_host)

## login to server
imap.login(imap_user, imap_pass)

imap.select('INBOX')


('OK', [b'19285'])

In [20]:
status, [data] = imap.search(None, '(ALL)')
#check
if status == "OK":
    messages = data.split()
    messages.reverse()
else:
    raise Exception


In [21]:
df = pd.DataFrame(columns=['msg_num','from','subject','body'])

for num in messages[0:500]:
	result, data = imap.fetch(num, '(RFC822)')
	print(f"Read message: {int(num)}")
	try:
		msg = email2Text(data[0][1])
	except (LookupError,TypeError):
		print(f"Skip message {int(num)}")
		continue

	if msg["body"] == []:
		body=''
	else:
		body=msg["body"][0]
	
	df.loc[int(num)] = [num,msg['from'],msg['subject'],clean_up_pipeline(body)]
	#break


Read message: 19285
Read message: 19284
Read message: 19283
Read message: 19282
Read message: 19281
Read message: 19280
Read message: 19279
Read message: 19278
Read message: 19277
Read message: 19276
Read message: 19275
Read message: 19274
Read message: 19273
Read message: 19272
Read message: 19271
Read message: 19270
Read message: 19269
Read message: 19268
Read message: 19267
Read message: 19266
Read message: 19265
Read message: 19264
Read message: 19263
Read message: 19262
Read message: 19261
Read message: 19260
Read message: 19259
Read message: 19258
Read message: 19257
Read message: 19256
Read message: 19255
Read message: 19254
Read message: 19253
Read message: 19252
Read message: 19251
Read message: 19250
Read message: 19249
Read message: 19248
Read message: 19247
Read message: 19246
Read message: 19245
Read message: 19244
Read message: 19243
Read message: 19242
Read message: 19241
Read message: 19240
Read message: 19239
Read message: 19238
Read message: 19237
Read message: 19236


In [22]:
df.reset_index(drop=True,inplace=True)
df


,msg_num,from,subject,body
0,b'19285',Luisa Mühle via XING <mailrobot@mail.xing.com>,Erinnerung: Unbeantwortete Recruiting-Nachricht,nachricht im browser ansehen xing hallo floria...
1,b'19284',Georg Behringer <georg@behringer-finanzmakler.de>,Neue Pressemitteilung von Dr. Michael Jaffé zu...,sehr geehrte prkundin sehr geehrter prkunde in...
2,b'19283',Discord <notifications@discord.com>,Du hast Nachrichten in World of M.U.L.E. verpa...,du hast neue nachricht 🍺general world of mule ...
3,b'19282',Alina Aybar via XING <mailrobot@mail.xing.com>,Alina Aybar möchte Dich als Kontakt hinzufügen.,nachricht im browser ansehen xing hallo floria...
4,b'19281',"""Noa @ CodiumAI"" <noa.g@codium.ai>",[CodiumAI] Teams plan launched! Your trial sta...,codiumai trial starts now hi florian were exci...
...,...,...,...,...
494,b'18790',Noa <noa.g@codium.ai>,[CodiumAI] September Recap 🟣🟢,we gathered a bunch of exciting updates and co...
495,b'18789',Badische Zeitung <abo@abonl.badische-zeitung.de>,"Herr Wolpert, die Badische Zeitung sagt Dankes...",unser geschenk für sie herr wolpert die badisc...
496,b'18788',Elias Becker via XING <mailrobot@mail.xing.com>,Hast Du diese Recruiting-Nachricht vielleicht ...,nachricht im browser ansehen xing hallo floria...
497,b'18787',GermanZero Spendenteam <spendenteam@germanzero...,Hilf uns auch weiterhin beim Kampf gegen die K...,liebe unterstützerinnen im juni hat das bundes...


In [23]:
features = cv.transform(df['body'])
pd.DataFrame(data=features.toarray(),columns=cv.get_feature_names_out())


,aacecefecalcdafbunptpcidemailreceiptxclick,aacecefeppidrtcnacderstadeded,aafcabcffefamptypetoguestwakeup,ab,abandoned,abbestell,abbuchung,abdeck,abdomax,abend,...,zweisprachig,zwischensumm,zwischenzeitlich,zwnj,zwpghkjkyrehmoxneusnnrupxgjfhhrkksdfbgasypdzlosdcxy,zxkywepatsdeweuqyibkwrttfoityqmpmfotqqdblmhubfsnopqwrkrbdxzvsqwvkiiuapdktmedbkrdmpkpzgrlhgsaacjfyzcilnyfc,zxqragjvppwwclhwwdwwbgpfsoenk,zxsweogdnrmswhdphdlcvimgsaernxlkqaabujwcfmqomvvtscvzvwacmqdo,zysvpvnkabfrbumduodefqgddcolugpkssefsjgtvtqeajcglwexgyuoeijbsctkntzfnxvyvhqsoinuklxtzivsdanjqxxhugktnzclzxavccutcbpq,å¾
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
495,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
cv.get_feature_names_out()


array(['aacecefecalcdafbunptpcidemailreceiptxclick',
       'aacecefeppidrtcnacderstadeded', 'aafcabcffefamptypetoguestwakeup',
       ...,
       'zxsweogdnrmswhdphdlcvimgsaernxlkqaabujwcfmqomvvtscvzvwacmqdo',
       'zysvpvnkabfrbumduodefqgddcolugpkssefsjgtvtqeajcglwexgyuoeijbsctkntzfnxvyvhqsoinuklxtzivsdanjqxxhugktnzclzxavccutcbpq',
       'å¾'], dtype=object)

In [25]:
predictions = model.predict(features)
predictions


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False,  True, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False,  True, False, False, False, False, False,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False,  True,  True, False, False, False, False,
       False, False,

In [26]:
df['label'] = pd.Series(predictions)
df


,msg_num,from,subject,body,label
0,b'19285',Luisa Mühle via XING <mailrobot@mail.xing.com>,Erinnerung: Unbeantwortete Recruiting-Nachricht,nachricht im browser ansehen xing hallo floria...,False
1,b'19284',Georg Behringer <georg@behringer-finanzmakler.de>,Neue Pressemitteilung von Dr. Michael Jaffé zu...,sehr geehrte prkundin sehr geehrter prkunde in...,False
2,b'19283',Discord <notifications@discord.com>,Du hast Nachrichten in World of M.U.L.E. verpa...,du hast neue nachricht 🍺general world of mule ...,False
3,b'19282',Alina Aybar via XING <mailrobot@mail.xing.com>,Alina Aybar möchte Dich als Kontakt hinzufügen.,nachricht im browser ansehen xing hallo floria...,False
4,b'19281',"""Noa @ CodiumAI"" <noa.g@codium.ai>",[CodiumAI] Teams plan launched! Your trial sta...,codiumai trial starts now hi florian were exci...,False
...,...,...,...,...,...
494,b'18790',Noa <noa.g@codium.ai>,[CodiumAI] September Recap 🟣🟢,we gathered a bunch of exciting updates and co...,True
495,b'18789',Badische Zeitung <abo@abonl.badische-zeitung.de>,"Herr Wolpert, die Badische Zeitung sagt Dankes...",unser geschenk für sie herr wolpert die badisc...,False
496,b'18788',Elias Becker via XING <mailrobot@mail.xing.com>,Hast Du diese Recruiting-Nachricht vielleicht ...,nachricht im browser ansehen xing hallo floria...,False
497,b'18787',GermanZero Spendenteam <spendenteam@germanzero...,Hilf uns auch weiterhin beim Kampf gegen die K...,liebe unterstützerinnen im juni hat das bundes...,False


## Select spam mails

In [27]:
df_spam = df[df['label']]
df_spam


,msg_num,from,subject,body,label
35,b'19250',Outdooractive <newsletter@news.outdooractive.com>,Discover content from official sources,discover official content only on outdooractiv...,True
38,b'19247',XING <info@e-mail.xing.com>,"Dein Profil wurde vor Kurzem besucht, Florian.",sicher dir auf xing premium und zahle nur € pr...,True
47,b'19238',"""Noa Green, Director of Customer Success"" <noa...",[CodiumAI] Your wish is our /command 🧞‍♂️,all our capabilities available with chat comma...,True
57,b'19228',"""Brandon Q. Morris"" <brandon@hardsf.de>",Schon gespannt auf Krill? 🚀,neuigkeiten von brandon q morris wie hängen ga...,True
69,b'19216',Almas from SpatialChat <almas@spatial.chat>,SpatialChat News Digest #8,spatialchat center spatialchat news digest by ...,True
...,...,...,...,...,...
486,b'18798',Cortexi <sec@nodecomfort.za.com>,"Tinnitus is a sign of losing Memory, Don't ign...",psckcjbdzsairsrburtwkgbffm this email was sent...,True
487,b'18797',Cortexi <updating@liftnorm.sa.com>,4 Stages of Memory Loss. Tinnitus is one!,lqkqdtpebgwdhehpmndgbgktze this email was sent...,True
488,b'18796',Alpilean <contaminated@cornerministry.sa.com>,Scientists discover a shocking New Cause of be...,september new scientific discovery stanford sc...,True
493,b'18791',Ritual <indecisiveness@saladsupport.sa.com>,Ancient Ritual That Will Boost Your Metabolism!,ritualjpg struggling with low energy and stubb...,True


## Flag emails that appeared to be spam

In [28]:
df_spam.apply(lambda x: imap.store(x['msg_num'],'FLAGS','\\Flagged $MailFlagBit1'),axis=1)


35     (OK, [b'19250 (FLAGS (\\Flagged $MailFlagBit1)...
38     (OK, [b'19247 (FLAGS (\\Flagged $MailFlagBit1)...
47     (OK, [b'19238 (FLAGS (\\Flagged $MailFlagBit1)...
57     (OK, [b'19228 (FLAGS (\\Flagged $MailFlagBit1)...
69     (OK, [b'19216 (FLAGS (\\Flagged $MailFlagBit1)...
                             ...                        
486    (OK, [b'18798 (FLAGS (\\Flagged $MailFlagBit1)...
487    (OK, [b'18797 (FLAGS (\\Flagged $MailFlagBit1)...
488    (OK, [b'18796 (FLAGS (\\Flagged $MailFlagBit1)...
493    (OK, [b'18791 (FLAGS (\\Flagged $MailFlagBit1)...
494    (OK, [b'18790 (FLAGS (\\Flagged $MailFlagBit1)...
Length: 104, dtype: object

In [29]:
imap.close()


('OK', [b'Close completed (0.093 + 0.000 + 0.092 secs).'])